# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.svm import SVC


nltk.download('wordnet') 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhide\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhide\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhide\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bhide\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("data",con = engine)
X = df.message 
y = df.drop(['id','message','original','genre'],axis =1) 

In [8]:
df['message'][df['related'] == 2]

Series([], Name: message, dtype: object)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """ Function to tokenize given text
        Args:
            text (str) : text to be tokenised
        Returns:
            words (str): tokenized text
    """
    
    # Normalization
    text = re.sub(r"[^a-zA-Z0-9]"," ", text.lower())
    #tokenization
    words = word_tokenize(text)
    # stop word removal
    words = [w for w in words if w not in stopwords.words("english")] 
    #Lemmatizing
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
            ('count_vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier()))
            ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
 X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.25)

In [6]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('count_vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f80118eca70>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
train_pred = pipeline.predict(X_train)
test_pred = pipeline.predict(X_test)

In [8]:
def classification_report(predcition, actual):
    """ Function to evaluate the classification performance per category
        Args:
            prediction (list) : prediction on test set
            actaul (list) : actual labels for test set
        Returns:
            evaluation (dict) : dictionary consisting of accuracy, precision, recall and f1 scores for the given classification task
    """
    acc = accuracy_score(predcition, actual)
    p = precision_score(predcition, actual, average='micro')
    r = recall_score(predcition, actual, average='micro')
    f1 = f1_score(predcition, actual, average='micro')
    evaluation = {"accuracy":np.round(acc,4), "precision":np.round(p,4), "recall" : np.round(r,4), "f1-score" : np.round(f1,4) }
    return evaluation

In [9]:
np.array(y_train)[:,2]

array([0, 0, 0, ..., 1, 0, 0])

In [10]:
## evlaution on train set
train_pred_eval = []

for i, columns in enumerate(y_train.columns):
    train_pred_eval.append( classification_report(train_pred[:,i], np.array(y_train)[:,i]))
train_eval_df = pd.DataFrame(train_pred_eval, index=y_train.columns)
train_eval_df.head() 

,accuracy,precision,recall,f1-score
related,0.9980,0.9980,0.9980,0.9980
request,0.9992,0.9992,0.9992,0.9992
offer,0.9998,0.9998,0.9998,0.9998
aid_related,0.9991,0.9991,0.9991,0.9991
medical_help,0.9997,0.9997,0.9997,0.9997


In [11]:
test_pred_eval = []

for i, columns in enumerate(y_test.columns):
    test_pred_eval.append( classification_report(test_pred[:,i], np.array(y_test)[:,i]))
test_eval_df = pd.DataFrame(test_pred_eval, index=y_test.columns)
test_eval_df.head()   

,accuracy,precision,recall,f1-score
related,0.8291,0.8291,0.8291,0.8291
request,0.8956,0.8956,0.8956,0.8956
offer,0.9959,0.9959,0.9959,0.9959
aid_related,0.7753,0.7753,0.7753,0.7753
medical_help,0.9216,0.9216,0.9216,0.9216


In [12]:
# mean scores for the predcition on test set
test_eval_df.mean()

accuracy     0.948175
precision    0.948175
recall       0.948175
f1-score     0.948175
dtype: float64

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters ={'clf__estimator__criterion':['gini'],
 'clf__estimator__max_features': ['auto', 'sqrt'],
 'clf__estimator__max_depth': [10, 20, 30, 40],
              'clf__estimator__random_state': [42]}


In [14]:
def score(y_true, y_pred):
    f1 = []
    for i,column in enumerate(y_train.columns):
        accuracy = f1_score(np.array(y_true)[:,i],y_pred[:,i], average = 'micro')
        f1.append(accuracy)
    avg_score = np.mean(f1)
    return avg_score

In [15]:
scr = make_scorer(score)

In [16]:
cv = GridSearchCV( pipeline, param_grid=parameters, scoring = scr, verbose= 4, return_train_score=True)

In [17]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END clf__estimator__criterion=gini, clf__estimator__max_depth=10, clf__estimator__max_features=auto, clf__estimator__random_state=42;, score=(train=0.928, test=0.926) total time= 1.7min
[CV 2/5] END clf__estimator__criterion=gini, clf__estimator__max_depth=10, clf__estimator__max_features=auto, clf__estimator__random_state=42;, score=(train=0.928, test=0.927) total time= 1.7min
[CV 3/5] END clf__estimator__criterion=gini, clf__estimator__max_depth=10, clf__estimator__max_features=auto, clf__estimator__random_state=42;, score=(train=0.928, test=0.928) total time= 1.7min
[CV 4/5] END clf__estimator__criterion=gini, clf__estimator__max_depth=10, clf__estimator__max_features=auto, clf__estimator__random_state=42;, score=(train=0.928, test=0.929) total time= 1.7min
[CV 5/5] END clf__estimator__criterion=gini, clf__estimator__max_depth=10, clf__estimator__max_features=auto, clf__estimator__random_state=42;, score=(train=0.9

GridSearchCV(estimator=Pipeline(steps=[('count_vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f80118eca70>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__criterion': ['gini'],
                         'clf__estimator__max_depth': [10, 20, 30, 40],
                         'clf__estimator__max_features': ['auto', 'sqrt'],
                         'clf__estimator__random_state': [42]},
             return_train_score=True, scoring=make_scorer(score), verbose=4)

In [18]:
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_depth': 40,
 'clf__estimator__max_features': 'auto',
 'clf__estimator__random_state': 42}

In [19]:
best = cv.best_estimator_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred_train = best.predict(X_train)
y_pred_test = best.predict(X_test)

In [21]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = classification_report(y_train.loc[:,column].values,y_pred_train[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df

,accuracy,precision,recall,f1-score
0,0.7733,0.7733,0.7733,0.7733
1,0.8783,0.8783,0.8783,0.8783
2,0.9960,0.9960,0.9960,0.9960
3,0.8192,0.8192,0.8192,0.8192
4,0.9278,0.9278,0.9278,0.9278
5,0.9563,0.9563,0.9563,0.9563
6,0.9726,0.9726,0.9726,0.9726
7,0.9817,0.9817,0.9817,0.9817
8,0.9727,0.9727,0.9727,0.9727
9,1.0000,1.0000,1.0000,1.0000


In [22]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = classification_report(y_test.loc[:,column].values,y_pred_test[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,accuracy,precision,recall,f1-score
0,0.7724,0.7724,0.7724,0.7724
1,0.8543,0.8543,0.8543,0.8543
2,0.9959,0.9959,0.9959,0.9959
3,0.7428,0.7428,0.7428,0.7428
4,0.9197,0.9197,0.9197,0.9197
5,0.9495,0.9495,0.9495,0.9495
6,0.9739,0.9739,0.9739,0.9739
7,0.9834,0.9834,0.9834,0.9834
8,0.9652,0.9652,0.9652,0.9652
9,1.0000,1.0000,1.0000,1.0000


In [23]:
import pickle
filename = 'best_rf.pkl'
pickle.dump(best, open(filename, 'wb'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [24]:

# using SVM instead 
pipe2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 = {'clf__estimator__learning_rate': [0.01, 0.02, 0.05],
              'clf__estimator__n_estimators': [10, 20, 40]}

cv2 = GridSearchCV(pipe2, param_grid = parameters2, scoring = scr, verbose = 4,return_train_score=True)

# Find best parameters
T_model2 = cv2.fit(X_train, y_train)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=10;, score=(train=0.937, test=0.935) total time= 1.5min
[CV 2/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=10;, score=(train=0.936, test=0.936) total time= 1.5min
[CV 3/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=10;, score=(train=0.936, test=0.936) total time= 1.5min
[CV 4/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=10;, score=(train=0.936, test=0.937) total time= 1.5min
[CV 5/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=10;, score=(train=0.936, test=0.936) total time= 1.5min
[CV 1/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=20;, score=(train=0.936, test=0.935) total time= 1.8min
[CV 2/5] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=20;, score=(train=0.936, test=0.936) total time= 1.8min
[C

In [25]:
cv2.best_params_

{'clf__estimator__learning_rate': 0.05, 'clf__estimator__n_estimators': 40}

In [26]:
best2 = cv2.best_estimator_

In [27]:
y_pred_train = best2.predict(X_train)
y_pred_test = best2.predict(X_test)

In [28]:
train_results = []
for i,column in enumerate(y_train.columns):
    result = classification_report(y_train.loc[:,column].values,y_pred_train[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df

,accuracy,precision,recall,f1-score
0,0.7581,0.7581,0.7581,0.7581
1,0.8538,0.8538,0.8538,0.8538
2,0.9954,0.9954,0.9954,0.9954
3,0.6654,0.6654,0.6654,0.6654
4,0.9219,0.9219,0.9219,0.9219
5,0.9509,0.9509,0.9509,0.9509
6,0.9723,0.9723,0.9723,0.9723
7,0.9816,0.9816,0.9816,0.9816
8,0.9679,0.9679,0.9679,0.9679
9,1.0000,1.0000,1.0000,1.0000


In [29]:
test_results = []
for i,column in enumerate(y_test.columns):
    result = classification_report(y_test.loc[:,column].values,y_pred_test[:,i])
    test_results.append(result)
test_results_df = pd.DataFrame(test_results)
test_results_df

,accuracy,precision,recall,f1-score
0,0.7629,0.7629,0.7629,0.7629
1,0.8514,0.8514,0.8514,0.8514
2,0.9959,0.9959,0.9959,0.9959
3,0.6617,0.6617,0.6617,0.6617
4,0.9202,0.9202,0.9202,0.9202
5,0.9496,0.9496,0.9496,0.9496
6,0.9739,0.9739,0.9739,0.9739
7,0.9834,0.9834,0.9834,0.9834
8,0.9654,0.9654,0.9654,0.9654
9,1.0000,1.0000,1.0000,1.0000


In [30]:
filename = 'best_adab.pkl'
pickle.dump(best2, open(filename, 'wb'))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.